# CapsNet

Source: https://github.com/jindongwang/Pytorch-CapsuleNet/blob/master/test_capsnet.py

In [1]:
import sys 
sys.path.insert(0, "helper")

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torchvision import datasets, transforms
import torchvision
from tqdm import tqdm

# =============================================================================
# datasceyence
# =============================================================================
from helper.visualisation.feature_map import *
from helper.model.capsnet import CapsNet

from helper.data.mnist import DataMNIST
from helper.data.retinamnist import DataRetinaMNIST
from helper.data.octmnist import DataOCTMNIST

#train_kwargs["device"] = True if torch.cuda.is_available() else False
#train_kwargs["batch_size"] = 5
# train_kwargs["epochs"] = 30
#LEARNING_RATE = 0.01
#MOMENTUM = 0.9

C:\Users\Prinzessin\anaconda3\envs\feta\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.25.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
seed = 1997 # was 19 before

torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

print("cuda available:", torch.cuda.is_available())

debug_model = False

print('torch 2.0.0 ==', torch.__version__=='2.0.0' , '->', torch.__version__)

cuda available: True
torch 2.0.0 == False -> 1.13.1


# Settings

In [3]:
model_kwargs = { # for 1 Channel
    'in_channels' : 1,
    'n_classes': None, # filled in the dataset
    'criterion': torch.nn.CrossEntropyLoss(),# torch.nn.BCEWithLogitsLoss(),
    'optimizer': "sgd", # sgd adamw
    'base_lr': 0.001,
    'min_lr' : 0.00001,
    'momentum' : 0.9,
    'lr_update' : 100,
    
    # CNN (cnn)
    'cnn_in_channels' : 1, # duplicate ...
    'cnn_out_channels' : 256,
    'cnn_kernel_size' : 9,

    # Primary Capsule (pc)
    'pc_num_capsules' : 8,
    'pc_in_channels' : 256,
    'pc_out_channels' : 32,
    'pc_kernel_size' : 9,
    'pc_num_routes' : 32 * 6 * 6,

    # Digit Capsule (dc)
    'dc_num_capsules' : 10,
    'dc_num_routes' : 32 * 6 * 6,
    'dc_in_channels' : 8,
    'dc_out_channels' : 16,

    # Decoder
    'input_width' : 28,
    'input_height' : 28,
}


train_kwargs = {
    'result_path': "examples/example_results", # "example_results/lightning_logs", # not in use??
    'exp_name': "debug_mnist", # must include oct or retina
    'load_ckpt_file' : 'version_18/checkpoints/epoch=4-unpruned=192-val_f1=0.12.ckpt', # "version_0/checkpoints/epoch=94-unpruned=1600-val_f1=0.67.ckpt", # 'version_94/checkpoints/epoch=26-step=1080.ckpt', # change this for loading a file and using "test", if you want training, keep None
    'epochs': 50, # including the pretrain epochs - no adding up
    'img_size' : 28, #168, # keep mnist at original size, training didn't work when i increased the size ... # MNIST/MedMNIST 28 × 28 Pixel
    'batch_size': 2, # 128, # the higher the batch_size the faster the training - every iteration adds A LOT OF comp cost
    'log_every_n_steps' : 4, # lightning default: 50 # needs to be bigger than the amount of steps in an epoch (based on trainset size and batchsize)
    'device': "cuda:0" if torch.cuda.is_available() else "cpu",
    'num_workers' : 0, # 18, # 18 for computer, 0 for laptop
    'train_size' : (2 * 50), # total or percentage
    'val_size' : (2 * 50), # total or percentage
    'test_size' : 10, # total or percentage - 0 for all
}

print("train kwargs", train_kwargs)
print("model kwargs", model_kwargs)

kwargs = {'train_kwargs':train_kwargs, 'model_kwargs':model_kwargs}


train kwargs {'result_path': 'examples/example_results', 'exp_name': 'debug_mnist', 'load_ckpt_file': 'version_18/checkpoints/epoch=4-unpruned=192-val_f1=0.12.ckpt', 'epochs': 50, 'img_size': 28, 'batch_size': 2, 'log_every_n_steps': 4, 'device': 'cuda:0', 'num_workers': 0, 'train_size': 100, 'val_size': 100, 'test_size': 10}
model kwargs {'in_channels': 1, 'n_classes': None, 'criterion': CrossEntropyLoss(), 'optimizer': 'sgd', 'base_lr': 0.001, 'min_lr': 1e-05, 'momentum': 0.9, 'lr_update': 100, 'cnn_in_channels': 1, 'cnn_out_channels': 256, 'cnn_kernel_size': 9, 'pc_num_capsules': 8, 'pc_in_channels': 256, 'pc_out_channels': 32, 'pc_kernel_size': 9, 'pc_num_routes': 1152, 'dc_num_capsules': 10, 'dc_num_routes': 1152, 'dc_in_channels': 8, 'dc_out_channels': 16, 'input_width': 28, 'input_height': 28}


# Data

In [4]:
if 'oct' in train_kwargs['exp_name']:
    # OCTMINST
    data = DataOCTMNIST(train_kwargs, model_kwargs)
        
elif 'retina' in train_kwargs['exp_name']:
    # RetinaMNIST
    data = DataRetinaMNIST(train_kwargs, model_kwargs)
    
else:
    # MNIST
    data = DataMNIST(train_kwargs, model_kwargs)
    
print(model_kwargs['n_classes'])

MNIST classes: ['0 - zero', '1 - one', '2 - two', '3 - three', '4 - four', '5 - five', '6 - six', '7 - seven', '8 - eight', '9 - nine']
10


# Run

## Lightning - should become lightning later on

In [5]:
def train(model, optimizer, train_loader, epoch):
    capsule_net = model
    capsule_net.train()
    n_batch = len(list(enumerate(train_loader)))
    total_loss = 0
    for batch_id, (data, target) in enumerate(tqdm(train_loader)):
        
        # one hot
        try:
            # print(target)
            # tensor([6, 5])
            target = torch.sparse.torch.eye(10).index_select(dim=0, index=target)
        except: 
            # print(target.squeeze(1))
            # tensor([6, 5])
            target = torch.sparse.torch.eye(10).index_select(dim=0, index=target.squeeze(1))
        
        # print(target)
        # tensor([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        # [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]])
        
        data, target = Variable(data), Variable(target)

        data, target = data.to(train_kwargs["device"]), target.to(train_kwargs["device"])

        optimizer.zero_grad()
        output, reconstructions, masked = capsule_net(data)
        
        path = f"examples/example_results/capsnet/rec_train_ep{epoch}_cl{np.argmax(target.data.cpu().numpy(), 1)[0]}.png"
        if (epoch % 5) == 0 and not(os.path.isfile(path)):
            torchvision.utils.save_image(reconstructions[0], path)
        elif epoch == 1:
            path = f"examples/example_results/capsnet/rec_train_ep{epoch}_id{batch_id}_cl{np.argmax(target.data.cpu().numpy(), 1)[0]}.png"
            torchvision.utils.save_image(reconstructions[0], path)
            
        
        loss = capsule_net.loss(data, output, target, reconstructions)
        loss.backward()
        optimizer.step()
        correct = sum(np.argmax(masked.data.cpu().numpy(), 1) == np.argmax(target.data.cpu().numpy(), 1))
        train_loss = loss.item()
        total_loss += train_loss
        if batch_id % 100 == 0:
            tqdm.write("Epoch: [{}/{}], Batch: [{}/{}], train accuracy: {:.6f}, loss: {:.6f}".format(
                epoch,
                train_kwargs["epochs"],
                batch_id + 1,
                n_batch,
                correct / float(train_kwargs["batch_size"]),
                train_loss / float(train_kwargs["batch_size"])
                ))
    tqdm.write('Epoch: [{}/{}], train loss: {:.6f}'.format(epoch,train_kwargs["epochs"],total_loss / len(train_loader.dataset)))


def val(capsule_net, val_dataloader, epoch):
    capsule_net.eval()
    val_loss = 0
    correct = 0
    for batch_id, (data, target) in enumerate(val_dataloader):
    
        # one hot
        try:
            # print(target)
            # tensor([6, 5])
            target = torch.sparse.torch.eye(10).index_select(dim=0, index=target)
        except: 
            # print(target.squeeze(1))
            # tensor([6, 5])
            target = torch.sparse.torch.eye(10).index_select(dim=0, index=target.squeeze(1))
        
        # print(target)
        # tensor([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        # [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]])
        
        data, target = Variable(data), Variable(target)

        data, target = data.to(train_kwargs["device"]), target.to(train_kwargs["device"])

        output, reconstructions, masked = capsule_net(data)
        
        """
        2 = batch size
        1 = channel
        28x28 = img size
        10 = classes
        16 = digit output channels 
        torch.Size([2, 10, 16, 1]) = output
        torch.Size([2, 1, 28, 28]) = reconstruction
        torch.Size([2, 10]) = masked - the thing we want for classification
        """
        
        path = f"examples/example_results/capsnet/rec_val_ep{epoch}_cl{np.argmax(target.data.cpu().numpy(), 1)[0]}.png"
        if (epoch % 5) == 0 and not(os.path.isfile(path)):
            torchvision.utils.save_image(reconstructions[0], path)
        
        #print(output.shape)
        #print(reconstructions.shape)
        #print(masked.shape)
        
        #print(output)
        #print(reconstructions)
        #print(masked)
        
        loss = capsule_net.loss(data, output, target, reconstructions)

        val_loss += loss.item()
        correct += sum(np.argmax(masked.data.cpu().numpy(), 1) ==
                       np.argmax(target.data.cpu().numpy(), 1))

    tqdm.write(
        "Epoch: [{}/{}], val accuracy: {:.6f}, loss: {:.6f}".format(epoch, train_kwargs["epochs"], correct / len(val_dataloader.dataset),
                                                                  val_loss / len(val_dataloader)))


## run training

In [6]:
capsule_net = CapsNet(model_kwargs)
capsule_net = torch.nn.DataParallel(capsule_net)

capsule_net = capsule_net.to(train_kwargs["device"])
capsule_net = capsule_net.module

optimizer = torch.optim.Adam(capsule_net.parameters())

for e in range(1, train_kwargs["epochs"] + 1):
    train(capsule_net, optimizer, data.train_dataloader, e)
    val(capsule_net, data.val_dataloader, e)

in_channels 1


  2%|█▋                                                                                 | 1/50 [00:04<03:35,  4.41s/it]

Epoch: [1/50], Batch: [1/50], train accuracy: 0.000000, loss: 0.449780


100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [00:22<00:00,  2.18it/s]


Epoch: [1/50], train loss: 0.417020
Epoch: [1/50], val accuracy: 0.500000, loss: 0.761273


...

Epoch: [49/50], train loss: 0.001196
Epoch: [49/50], val accuracy: 0.620000, loss: 0.501107


  2%|█▋                                                                                 | 1/50 [00:00<00:17,  2.82it/s]

Epoch: [50/50], Batch: [1/50], train accuracy: 1.000000, loss: 0.000139


100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [00:19<00:00,  2.55it/s]


Epoch: [50/50], train loss: 0.001031
Epoch: [50/50], val accuracy: 0.670000, loss: 0.523572
